This short introduction uses Keras to:

- Build a neural network that classifies images.
- Train this neural network.
- And, finally, evaluate the accuracy of the model.

In [1]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

Build the [tf.keras.Sequential] model by stacking layers. Choose an optimizer and loss function for training:

In [3]:
import numpy as np

x_train_show = np.array(x_train)
y_train_show = np.array(y_train)
x_test_show = np.array(x_test)
y_test_show = np.array(y_test)

print("shape of x_train : ", x_train_show.shape)
print("shape of y_train : ", y_train_show.shape)
print("shape of x_test  : ", x_test_show.shape)
print("shape of y_test  : ", y_test_show.shape)

# Now I can conclude if x was a data and y was a label

shape of x_train :  (60000, 28, 28)
shape of y_train :  (60000,)
shape of x_test  :  (10000, 28, 28)
shape of y_test  :  (10000,)


In [4]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

For each example the model returns a vector of [logits](https://developers.google.com/machine-learning/glossary#logits) or [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) scores, one for each class.

In [5]:
predictions = model(x_train[:1]).numpy()
predictions

array([[  40.079464 ,    4.8291435,  -32.56197  ,  -62.58025  ,
          17.397354 , -175.54837  ,  167.14836  ,  -31.87657  ,
         101.334335 ,  -64.56077  ]], dtype=float32)

The [tf.nn.softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) function converts these logits to "probabilities" for each class:

In [6]:
tf.nn.softmax(predictions).numpy()

array([[0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        2.6141574e-29, 0.0000000e+00]], dtype=float32)

**Note**: It is possible to bake this [tf.nn.softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax) in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to provide an exact and numerically stable loss calculation for all models when using a softmax output.

The [losses.SparseCategoricalCrossentropy](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) loss takes a vector of logits and a True index and returns a scalar loss for each example.

In [7]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class: It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [8]:
loss_fn(y_train[:1], predictions).numpy()

342.69672

In [9]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The [Model.fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) method adjusts the model parameters to minimize the loss:

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 2ms/step - loss: 2.3788 - accuracy: 0.7481
Epoch 2/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.5551 - accuracy: 0.8511
Epoch 3/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4602 - accuracy: 0.8783
Epoch 4/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4074 - accuracy: 0.8897
Epoch 5/5
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3944 - accuracy: 0.8954


The [Model.evaluate](https://www.tensorflow.org/api_docs/python/tf/keras/Model#evaluate) method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.2766 - accuracy: 0.9361


[0.276576429605484, 0.9361000061035156]